In [22]:
import ee
import geemap

from eelib import collections

In [23]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [52]:
def maskS2Clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 2 << 11
    
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    return image.updateMask(mask)

# Adds Earth Engine dataset
image = ee.ImageCollection("projects/fpca-336015/assets/williston-cba").geometry().dissolve()


# Sentinel - 1 
# s1 = collections.S1Collection().filterBounds(image).filter('relativeOrbitNumber_start == 64').filterDate('2018-05-01', '2018-06-01').geometry().dissolve()

# Intersect of Data Cube and Sentinel - 1
intersection = s1.intersection(image)

# Sets visualization parameters.
vis_params = {
        'min': 0,
        'max': 4000,
        'bands': ['B4', 'B3', 'B2']}


# Adds Earth Engine layers to Map
Map.addLayer(s1, {}, 'SRTM DEM')
Map.addLayer(image, {}, 'Geometry')
Map.addLayer(intersection, {}, 'intersection')
# Map.centerObject(xy, 13)

# Display the Map
Map.addLayerControl()

In [26]:
isinstance(intersection, ee.Geometry)

True

In [55]:
s2 = collections.S2TOACollection().filterBounds(intersection).filterDate('2018-06-21', '2018-09-20').filter('CLOUDY_PIXEL_PERCENTAGE < 20').sort('CLOUDY_PIXEL_PERCENTAGE')
print(s2.size().getInfo())
s2 = s2.map(maskS2Clouds)

264


In [56]:
Map.addLayer(s2, vis_params, 'Sentinel - 2')
Map.addLayer(intersection, {}, 'intersection')